In [1]:
from IPython.display import display, HTML
from bs4 import BeautifulSoup
from urllib import parse
from time import sleep
from datetime import datetime
from multiprocessing.pool import ThreadPool
import pandas as pd
import requests

In [2]:
SYMBOL = "AMZN"

INITIAL_URL = "https://www.nasdaq.com/symbol/{}/institutional-holdings".format(SYMBOL.lower())

In [3]:
def get_page_soup_object(url, key="id", value=None, sub_object_name=""):
    """
    a helper function that loads and parses html pages
    sometimes we get blocked or other errors, so we will check if elements we need are on the page
    """
    retries, max_retries = 0, 4
    while retries < max_retries:
        try:
            page = requests.get(url)
        except Exception as e:
            print(e)
            element = None
        else:
            soup = BeautifulSoup(page.content, 'html.parser')
            # check elements on the page
            element = soup.find(**{key: value})
        
        if element is None or sub_object_name and getattr(element, sub_object_name) is None:
            print("{}:{}.{} isn't on the page".format(key, value, sub_object_name))
            retries += 1
            sleep(retries * 2)  
        else:
            if sub_object_name:
                return getattr(element, sub_object_name)
            return element
    else:
        raise Exception("Cannot access data on the page. You must've been blocked.".format())

### 1. Let's start from the holders page

<img src="n18/1.png" width=800 />

In [4]:
# knowing the stucture, we can implement function that returns all the funders with their names, links and shares

def get_holders(holders_url):
    """
    a generator function
    returns: holder name, holder link, shares number
    """
    holders_block_table = get_page_soup_object(holders_url, "id", "quotes_content_left_pnlInsider", "table")
    for tr in holders_block_table.find_all("tr"): # iterate through the rows
        cells = [el for el in tr.children if el.name is not None]  # there non-tags(like \n) we need to filter out
        name_cell, date_cell, shares_cell, *_, value_cell = cells  # unpacking cell into distinct variables

        if name_cell.name == "th":
            continue  # it's the header, we don't need to proceed any further
        
        data = dict(
            Name=name_cell.a.string,
            DetailURL=name_cell.a["href"],
            SharesHeld=None if shares_cell.string is None else int(shares_cell.string.replace(",", "")),
            Value=None if value_cell.string is None else int(value_cell.string.replace(",", ""))
        )
        yield data

In [5]:
for e in get_holders(INITIAL_URL):
    print(e)

{'Value': 51841985, 'DetailURL': 'https://www.nasdaq.com/quotes/institutional-portfolio/vanguard-group-inc-61322', 'Name': 'VANGUARD GROUP INC', 'SharesHeld': 28527398}
{'Value': 45808388, 'DetailURL': 'https://www.nasdaq.com/quotes/institutional-portfolio/blackrock-inc-711679', 'Name': 'BLACKROCK INC.', 'SharesHeld': 25207255}
{'Value': 31118613, 'DetailURL': 'https://www.nasdaq.com/quotes/institutional-portfolio/fmr-llc-12407', 'Name': 'FMR LLC', 'SharesHeld': 17123825}
{'Value': 27710898, 'DetailURL': 'https://www.nasdaq.com/quotes/institutional-portfolio/price-t-rowe-associates-inc-md-2145', 'Name': 'PRICE T ROWE ASSOCIATES INC /MD/', 'SharesHeld': 15248641}
{'Value': 27518320, 'DetailURL': 'https://www.nasdaq.com/quotes/institutional-portfolio/state-street-corp-6697', 'Name': 'STATE STREET CORP', 'SharesHeld': 15142670}
{'Value': 17857129, 'DetailURL': 'https://www.nasdaq.com/quotes/institutional-portfolio/capital-world-investors-767199', 'Name': 'CAPITAL WORLD INVESTORS', 'Shares

### 2. Details page

The stats can be get from the table with id = "position-stats" <img src="n18/3.png"  width=600 />

In [6]:
def get_position_stats(url):
    """
    returns: total positions and total value
    """
    stats_table = get_page_soup_object(url, "id", "position-stats")
    
    rows = [el for el in stats_table.children if el.name == "tr"]
    first_cell, *_, last_cell = rows
    
    positions_string = first_cell.find(align="right").string
    value_string = last_cell.find(align="right").string
    
    positions = None if positions_string is None else int(positions_string.replace(",", ""))
    value = None if value_string is None else int(value_string.replace(",", ""))
    
    return positions, value
    

get_position_stats("https://www.nasdaq.com/quotes/institutional-portfolio/vanguard-group-inc-61322")

(4079, 2422250)

### There are about 160 pages * (15 + 1 links)  = 2560 requests, we have to run them in parallel

In [7]:
# we need a single function to run
def get_holders_data(url):
    results = []
    for data in get_holders(url):
        positions, total_value = get_position_stats(data["DetailURL"])
        data.update(
            TotalPositions=positions,
            TotalMktValue=total_value,
        )
        results.append(data)
    
    return results

We can use last link from the pager to compose all the pages links. 

<img src="n18/2.png" width=700 >

In [8]:
last_link = get_page_soup_object(INITIAL_URL, "id", "quotes_content_left_lb_LastPage")
parse_result = parse.urlparse(last_link["href"])
query_params = dict(parse.parse_qsl(parse_result.query))
query_params

{'page': '161'}

In [9]:
pool = ThreadPool(20)
all_pages_links = ("{}?page={}".format(INITIAL_URL, num) for num in range(1, int(query_params.get("page", 1)) + 1))
results = pool.map(get_holders_data, all_pages_links)

results[0][0]

id:quotes_content_left_pnlInsider.table isn't on the page
id:position-stats. isn't on the page
id:quotes_content_left_pnlInsider.table isn't on the page
id:position-stats. isn't on the page
id:position-stats. isn't on the page
id:quotes_content_left_pnlInsider.table isn't on the page
id:position-stats. isn't on the page


{'DetailURL': 'https://www.nasdaq.com/quotes/institutional-portfolio/vanguard-group-inc-61322',
 'Name': 'VANGUARD GROUP INC',
 'SharesHeld': 28527398,
 'TotalMktValue': 2422250,
 'TotalPositions': 4079,
 'Value': 51841985}

In [21]:
df = pd.DataFrame.from_records([row for page in results for row in page])
df

,DetailURL,Name,SharesHeld,TotalMktValue,TotalPositions,Value
0,https://www.nasdaq.com/quotes/institutional-po...,VANGUARD GROUP INC,28527398,2422250.0,4079,51841985.0
1,https://www.nasdaq.com/quotes/institutional-po...,BLACKROCK INC.,25207255,2204168.0,4813,45808388.0
2,https://www.nasdaq.com/quotes/institutional-po...,FMR LLC,17123825,901406.0,2658,31118613.0
3,https://www.nasdaq.com/quotes/institutional-po...,PRICE T ROWE ASSOCIATES INC /MD/,15248641,672902.0,2568,27710898.0
4,https://www.nasdaq.com/quotes/institutional-po...,STATE STREET CORP,15142670,1240282.0,3670,27518320.0
...,...,...,...,...,...,...
2409,https://www.nasdaq.com/quotes/institutional-po...,"W.E. DONOGHUE & CO., LLC",0,1209.0,192,NaN
2410,https://www.nasdaq.com/quotes/institutional-po...,WASATCH ADVISORS INC,0,8434.0,319,NaN
2411,https://www.nasdaq.com/quotes/institutional-po...,WILLOW CREEK WEALTH MANAGEMENT INC.,0,179.0,74,NaN
2412,https://www.nasdaq.com/quotes/institutional-po...,WOODMONT INVESTMENT COUNSEL LLC,0,366.0,194,NaN


In [22]:
df["RelativePosition"] = df.Value / df.TotalMktValue / 1000 # we  add "/1000", cos total value in millons and value in 1,000s
df = df.sort_values(by="RelativePosition", ascending=False).reset_index(drop=True)
del df["DetailURL"]
output = df.to_html(formatters={
    'Name': '{}'.format,
    'SharesHeld': '{:,}'.format,
    'TotalPositions': '{:,}'.format,
    'TotalMktValue': '{:,}'.format,
    'RelativePosition': '{:,.2%}'.format
})
display(HTML(output))

,Name,SharesHeld,TotalMktValue,TotalPositions,Value,RelativePosition
0,DGH INVESTMENTS INC.,"4,150",8.0,2,7542.0,94.27%
1,"URSA FUND MANAGEMENT, LLC","557,200","1,267.0",14,1012583.0,79.92%
2,"WORM CAPITAL, LLC","34,110",129.0,4,61987.0,48.05%
3,ARROWGRASS CAPITAL PARTNERS LLP,"65,200",253.0,10,118486.0,46.83%
4,SUMWAY DEVELOPMENT LTD.,"116,191",540.0,12,211150.0,39.10%
5,HUNTINGTON STEELE LLC,"83,615",437.0,74,151951.0,34.77%
6,SOCIAL CAPITAL PEP MANAGEMENT LLC,"22,211",118.0,6,40363.0,34.21%
7,YONG RONG (HK) ASSET MANAGEMENT LTD,"35,500",193.0,15,64513.0,33.43%
8,"JW ASSET MANAGEMENT, LLC","19,627",125.0,24,35668.0,28.53%
9,ANTIPODEAN ADVISORS LLC,"15,100",103.0,19,27441.0,26.64%
